In [1]:
import pandas as pd
import numpy as np
import os, sys
import re
import phylopandas as ph

In [2]:
## parse seq data
from Bio import SeqIO

In [3]:
#make a function to create subsample
## df has 3 column, sequence taxa, date, and country; here I group the dataset by year and country, n is the number for each group
#he output is a csv file which has a list of sampled taxa

def subsample(df,n,file):
    #Convert 'Date' to a datetime datatype.
    df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
    sample = df.groupby(['country',df['date'].dt.year],group_keys=False).apply(lambda x:x.sample(min(len(x),n)))
    taxa = sample['taxa']
    ## write taxa to a file
    taxa.to_csv(file, header = False, index =False)
    
    

In [40]:
subsample(RSVB,5,'RSVB_sample_1.txt')
subsample(RSVB,5,'RSVB_sample_2.txt')
subsample(RSVB,5,'RSVB_sample_3.txt')

In [5]:
## random sample
def random_sample(df,n,file):
    sample = df.sample(frac=n)
    taxa = sample['taxa']
    taxa.to_csv(file, header = False, index =False)
    


In [6]:
random_sample(RSVB,0.5,'RSVB_sample_1.txt')
random_sample(RSVB,0.5,'RSVB_sample_2.txt')
random_sample(RSVB,0.5,'RSVB_sample_3.txt')

In [ ]:
## Vaccine efficacy evaluation 
## RSVB_F, start with the orginal dataset
# from fasta file to get the year and country information
df = ph.read_fasta(fasta_file)
df = df.filter(['id','sequence'], axis=1)
df[['Accession','subtype',"country","date","genotype_ref"]] = df.id.str.split("-",expand=True) 

meta = pd.read_csv(meta_file)
meta = meta.filter(['Accession_x','collection_date_guess','genotype'], axis=1)
#meta
fasta=pd.merge(df, meta, how='left', left_on='Accession', right_on='Accession_x')
fasta=fasta.dropna(subset=['collection_date_guess'])
fasta["time"] = pd.to_datetime(fasta["collection_date_guess"])

In [15]:
df = ph.read_fasta("RAxML/RAxML_date/RAxML_date_RSVA_F/HRSVA_F_date.fasta")
df = df.filter(['id','sequence'], axis=1)
df[['Accession','subtype',"country","date","genotype_ref"]] = df.id.str.split("-",expand=True) 
#df['date2'] = pd.to_datetime(df['date'],infer_datetime_format=True)
#df["year"]=df.date2.dt.year
df

ValueError: Columns must be same length as key

In [43]:
df.groupby(['country', 'year']).size()

country         year
Argentina       2006     1
                2007     1
                2008     1
                2015     2
                2016     2
                        ..
United_Kingdom  2011     3
                2012    11
                2013     8
                2014     2
                2016     2
Length: 141, dtype: int64

In [44]:
sample = df.groupby(['country',df['date2'].dt.year],group_keys=False).apply(lambda x:x.sample(min(len(x),3)))

In [45]:
sample

,id,sequence,Accession,subtype,country,date,genotype_ref,date2,year
67,KF826839-B-Argentina-7/25/2006-BA,ATGGAGTTGCTGATCCATAGATCAAGTGCAATATTCCTAACTCTTG...,KF826839,B,Argentina,7/25/2006,BA,2006-07-25,2006
68,KF826842-B-Argentina-5/22/2007-BA,ATGGAGTTGCTGATCCATAGATCAAGTGCAATCTTCCTAACTCTTG...,KF826842,B,Argentina,5/22/2007,BA,2007-05-22,2007
71,KF826845-B-Argentina-6/10/2008-BA,ATGGAGTTGCTGATCCATAGATCAAGTGCAATCTTCCTAACTCTTG...,KF826845,B,Argentina,6/10/2008,BA,2008-06-10,2008
467,KY883568-B-Argentina-2/23/2015-BA,ATGGAGTTGCTGATCCATAGATCAAGTGCAATCTTCCTAACTCTTG...,KY883568,B,Argentina,2/23/2015,BA,2015-02-23,2015
252,KY883571-B-Argentina-5/26/2015-BA,ATGGAGTTGCTGATCCATAGATCAAGTGCAATCTTCCTAACTCTTG...,KY883571,B,Argentina,5/26/2015,BA,2015-05-26,2015
...,...,...,...,...,...,...,...,...,...
209,KY249662-B-United_Kingdom-11/19/2013-BA,ATGGAGTTGCTGATCCATAGATCAAGCGCAATCTTCCTAACTCTTG...,KY249662,B,United_Kingdom,11/19/2013,BA,2013-11-19,2013
411,KY249657-B-United_Kingdom-1/3/2014-BA,ATGGAGTTGCTGATCCATAGATCAAGTGCAATCTTCCTAACTCTTG...,KY249657,B,United_Kingdom,1/3/2014,BA,2014-01-03,2014
410,KY249656-B-United_Kingdom-1/7/2014-BA,ATGGAGTTGCTGATCCATAGATCAAGTGCAATCTTCCTAACTCTTG...,KY249656,B,United_Kingdom,1/7/2014,BA,2014-01-07,2014
420,KY249683-B-United_Kingdom-1/4/2016-BA,ATGGAGTTGCTGATCCATAGATCAAGTGCAATCTTCCTAACTCTTG...,KY249683,B,United_Kingdom,1/4/2016,BA,2016-01-04,2016


In [46]:
fasta=sample
fasta['taxa']=fasta.apply(lambda x:'%s-%s-%s-%s-%s-%s' % (x['Accession'],x['subtype'],x['year'],x['country'],x['date'],x['genotype_ref']),axis=1)
fasta=fasta.filter(['taxa','sequence'], axis=1)
#csv to fasta
fasta.to_csv("temp.tab", sep="\t",index = False,header=False)
fasta = SeqIO.parse("temp.tab", "tab")
SeqIO.write(fasta,"RSVB_F_epicc_subsample1019", "fasta")
os.remove("temp.tab")

In [10]:
## wrap into a function
def epicc_sample(original,n,sampleout ):
    df = ph.read_fasta(original)
    df = df.filter(['id','sequence'], axis=1)
    df[['Accession','subtype',"country","date","genotype_ref"]] = df.id.str.split("-",expand=True) 
    df['date2'] = pd.to_datetime(df['date'],infer_datetime_format=True)
    df["year"]=df.date2.dt.year
    print(df.groupby(['country', 'year']).size())
    sample = df.groupby(['country',df['date2'].dt.year],group_keys=False).apply(lambda x:x.sample(min(len(x),n)))
    ## from sample to fasta
    fasta=sample
    fasta['taxa']=fasta.apply(lambda x:'%s-%s-%s-%s-%s-%s' % (x['Accession'],x['subtype'],x['year'],x['country'],x['date'],x['genotype_ref']),axis=1)
    fasta=fasta.filter(['taxa','sequence'], axis=1)
    #csv to fasta
    fasta.to_csv("temp.tab", sep="\t",index = False,header=False)
    fasta = SeqIO.parse("temp.tab", "tab")
    SeqIO.write(fasta,sampleout, "fasta")
    os.remove("temp.tab")
    

In [26]:
epicc_sample("RAxML/RSVB_G_datev2/HRSVB_G_datev2_clean.fasta",3,"RSVB_G_epicc_subsample1112.fasta")

country         year
Argentina       2006     1
                2007     1
                2008     1
                2015     1
                2016     5
                        ..
United_Kingdom  2012    11
                2013     7
                2014     2
                2016     3
Viet_Nam        2009     2
Length: 150, dtype: int64


In [22]:
epicc_sample("RAxML/RAxML_date/RAxML_date_RSVA_F/HRSVA_F_date_clean.fasta",3,"RSVA_F_epicc_subsample1112.fasta")

country    year
Argentina  2004     2
           2005     1
           2006     1
           2007     1
           2008     1
                   ..
USA        2018     3
           2019     6
Uruguay    1992     1
Viet_Nam   2009     3
           2010    17
Length: 172, dtype: int64


In [25]:
epicc_sample("RAxML/RSVA_G_date2/HRSVA_G_date2_clean.fasta",3,"RSVA_G_epicc_subsample1112.fasta")

country    year
Argentina  2004     2
           2005     1
           2006     1
           2007     1
           2008     1
                   ..
USA        2013    49
           2014     7
           2016    15
Viet_Nam   2009     3
           2010    15
Length: 168, dtype: int64
